# Project 5: Satellite Trajectory Analysis GUI Solution 🛰️

## Objectives 
- Understand the contextual background behind satellite trajectories
- Create a GUI using `tkinter` and `matplotlib` to plot satellite details
- Make the GUI interactive by adding numerical sliders and buttons
- Allow the user to download a CSV file including data for the project

## Step 1: Import Libraries

In [1]:
import tkinter 
from tkinter import ttk
import sv_ttk
import csv
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.figure import Figure 
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,  
NavigationToolbar2Tk) 
from PyAstronomy import pyasl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sgp4.api import Satrec
from sgp4.api import jday

## Step 2: Create tkinter Window
Create a tkinter window titled "Satellite Trajectory Analysis" with dimensions of `1200x800`

In [3]:
root = tkinter.Tk()
root.title("Satellite Trajectory Analysis")
root.geometry('1200x800')

''

## Step 3: Configure Styling
1. Set the tkinter theme to Sun Valley (HINT: use `sv_ttk`)
2. Set the matplotlib theme to `dark_background` (HINT: use `plt.style.use`)
3. Create a tkinter `Style` instance
4. Create a tkinter class titled `Margin.TLabel` with the styles: `padding=(0, 20, 0, 5)`
5. Create a tkinter class titled `TButton` with the styles: `padding=(10, 10), background="#000"`

In [ ]:
# Set the theme to dark mode!
sv_ttk.set_theme("dark")
plt.style.use('dark_background')

# Create a Style instance to make custom styles!
style = ttk.Style()

style.configure('Margin.TLabel', padding=(0, 20, 0, 5))
style.configure('TButton', padding=(10, 10), background="#000") 

## Step 4: Creating Side-by-Side Frames
Tkinter allows us to create horizontally adjacent frames which allow us to visually seperate our components into a "Left Frame" and "Right Frame"

In [2]:
 # Create frames
left_frame = tkinter.Frame(root, width=500, height=600)
right_frame = tkinter.Frame(root, width=300, height=600)

# Pack frames side by side
left_frame.pack(side="left", fill="both", expand=True)
right_frame.pack(side="left", fill="both", expand=True)

NameError: name 'root' is not defined

## Step 5: Create Sliders for Keplarian Elements
Our interactive GUI will contain six sliders for each Keplarian Element.
The below code has three sliders: semi-major axis, eccentricity, and inclination.
Your task is to create three more sliders for the following: `Right Ascension of Ascending Node`, `Argument of Perigee`, and `Mean Anomaly`

In [4]:
semi_major_axis_label = ttk.Label(right_frame, text="Semi-Major Axis: ", style='Margin.TLabel')
semi_major_axis_label.pack()
semi_major_axis_slider = tkinter.Scale(right_frame, from_=2000, to=50000, tickinterval=10000, orient=tkinter.HORIZONTAL, length=400)
semi_major_axis_slider.pack()

eccentricity_label = ttk.Label(right_frame, text="Eccentricity: ", style='Margin.TLabel')
eccentricity_label.pack()
eccentricity_slider = tkinter.Scale(right_frame, from_=0, to=1, resolution=0.01, tickinterval=0.1, orient=tkinter.HORIZONTAL, length=400)
eccentricity_slider.pack()

inclination_label = ttk.Label(right_frame, text="Inclination: ", style='Margin.TLabel')
inclination_label.pack()
inclination_slider = tkinter.Scale(right_frame, from_=0, to=360, tickinterval=40, orient=tkinter.HORIZONTAL, length=400)
inclination_slider.pack()

raan_label = ttk.Label(right_frame, text="Right Ascension of Ascending Node: ", style='Margin.TLabel')
raan_label.pack()
raan_slider = tkinter.Scale(right_frame, from_=0, to=360, tickinterval=40, orient=tkinter.HORIZONTAL, length=400)
raan_slider.pack()

argument_periapsis_label = ttk.Label(right_frame, text="Argument of Perigee: ", style='Margin.TLabel')
argument_periapsis_label.pack()
argument_periapsis_slider = tkinter.Scale(right_frame, from_=1, to=360, tickinterval=40, orient=tkinter.HORIZONTAL, length=400)
argument_periapsis_slider.pack()

mean_anamoly_label = ttk.Label(right_frame, text="Mean Anomaly: ", style='Margin.TLabel')
mean_anamoly_label.pack()
mean_anamoly_slider = tkinter.Scale(right_frame, from_=1, to=360, tickinterval=40, orient=tkinter.HORIZONTAL, length=400)
mean_anamoly_slider.pack()

NameError: name 'right_frame' is not defined

In [6]:
# Takes in six Keplarian Elements as parameters and sets the value to the sliders in the GUI
def display_sliders(a, e, i, o, w, v):
    semi_major_axis_slider.set(a)
    eccentricity_slider.set(e)
    inclination_slider.set(i)
    raan_slider.set(o)
    argument_periapsis_slider.set(w)
    mean_anamoly_slider.set(v)

In [7]:
# Default:
# Semi-Major Axis: 10000
# Eccentricity: 0.1
# Inclination: 90
# RAAN: 40
# Argument of Periapsis: 1
# Mean Anamoly: 1
display_sliders(10000, 0.1, 90, 40, 1, 1)

NameError: name 'semi_major_axis_slider' is not defined

## Step 5: Create Altitude vs Time Graph
The following code uses the SGP4 propagation method to predict the satellite's altitude in respect to the surface of the Earth. This utilizes the `sgp4` method within the `Satrec` class provided by the `sgp4` library. 

The function takes in two lines of a TLE as inputs and finds the satellite position within a specified timeframe `times`. Then for each timestamp, it calculates the satellite's altitude above Earth using SGP4. 

In [9]:
def trace_altitude_graph(tle_one, tle_two):
    satellite = Satrec.twoline2rv(
    tle_one, tle_two
    )

    # Defining the time range
    start_time = 0
    end_time = 24 * 3600  # 1 day
    step = 60  # 1 minute
    times = np.arange(start_time, end_time, step)
    times = np.linspace(start_time, end_time, step)
    
    # Calculate the altitude at each time step
    altitudes = []
    for t in times:
        jd, fr = jday(2024, 4, 1, 0, 0, t)
        # Use SPG4 to get the position details of a satellite
        e, r, v = satellite.sgp4(jd, fr)
        # r represents the position vector of the satellite where:
        # [x, y, z]
        # Calculates the altitude of the satellite above the Earth's surface
        altitude = (r[0]**2 + r[1]**2 + r[2]**2)**0.5 - 6378.135  # Earth's mean radius in kilometers
        altitudes.append(altitude)

    # Intialize a figure and embed it within a canvas
    fig = Figure(figsize = (6, 3), dpi = 100) 
    canvas = FigureCanvasTkAgg(fig, master = left_frame) 
    canvas.get_tk_widget().pack(pady=15)
    plot = fig.add_subplot(111)

    # Plot timestamps and altitude
    plot.plot(times, altitudes)
    plot.grid(True)

    # Draw the plot
    canvas.draw() 

In [ ]:
# Default TLE:
# 1 25544U 98067A   21257.91276829  .00000825  00000-0  24323-4 0  9990
# 2 25544  51.6461  89.6503 0003031 120.4862 259.0942 15.4888108230711
trace_altitude_graph("1 25544U 98067A   21257.91276829  .00000825  00000-0  24323-4 0  9990", "2 25544  51.6461  89.6503 0003031 120.4862 259.0942 15.48881082307117")

## Step 6: Visualizing a 3D Orbit
The following creates a second figure displaying a 3D plot of a satellite's trajectory relative to Earth using the `pyasl` library. 
It extracts the positional trajectory data using the `KeplerEllipse` and `xyzPos` features of `pyasl`

In [10]:
# Intialize the second figure and canvas and place it within the left frame
fig2 = Figure(figsize = (6, 3), dpi = 100) 
canvas2 = FigureCanvasTkAgg(fig2, master = left_frame) 

# This method takes the six Keplarian Elements and plots them in a 3D graph
def visualize_3d_orbit(a, p, e, o, i, w, fig_vis):
    orbit = pyasl.KeplerEllipse(a=a, per=p, e=e, Omega=o, i=i, w=w)
    t = np.linspace(0, 4, 300)
    pos = orbit.xyzPos(t)

    # Clear the figure if it already exists (allows us to refresh / redraw updated plots)
    if fig_vis:
        fig_vis.clear()
    
    canvas2.get_tk_widget().pack(pady=15)
    plot2 = fig_vis.add_subplot(111, projection='3d')

    # Plots the Earth, trajectory path, and periapsis point of trajectory
    plot2.plot(0, 0, 'bo', markersize=9, label="Earth")
    plot2.plot(pos[::, 1], pos[::, 0], 'k-', label="Satellite Trajectory")
    plot2.plot(pos[0, 1], pos[0, 0], 'g*', label="Periapsis")

    # Draws the plot onto the canvas
    canvas2.draw() 

NameError: name 'left_frame' is not defined

In [11]:
visualize_3d_orbit(1.0, 1.0, 0.5, 0.0, 30.0, 0.0, fig2)

NameError: name 'visualize_3d_orbit' is not defined

## Step 7: Create a Retrace Orbit Button
1. Create a function titled `retrace_orbit` that invokes `visualize_3d_orbit`
with the values from each of the sliders. (HINT: to get the semi-major axis value
`semi_major_axis_slider.get()`)
2. Create a tkinter Button titled "Retrace Orbit" that calls `retrace_orbit` when it is clicked and place it within `right_frame`

In [12]:
def retrace_orbit():
    visualize_3d_orbit(semi_major_axis_slider.get(), mean_anamoly_slider.get(), eccentricity_slider.get(), raan_slider.get(), inclination_slider.get(), argument_periapsis_slider.get(), fig2)

In [ ]:
trace_button = ttk.Button(right_frame, text="Retrace Orbit", command=retrace_orbit, style='TButton')
trace_button.pack()

## Step 8: Create Download CSV Button
1. Create a function `download_csv` that creates a file `data.csv` with field names `["A", "E", "I", "O", "W", "V"]` using `csv.DictWriter` 
2. Within the function, write a header and a single row containing the slider values for each of the 6 Keplarian elements.
3. Create a button titled "Download CSV" that calls `download_csv` when clicked and apply the style `TButton`

In [13]:
def download_csv():
    with open('data.csv', 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=["A", "E", "I", "O", "W", "V"])
        writer.writeheader()
        writer.writerow({
            "A": semi_major_axis_slider.get(), 
            "E": eccentricity_slider.get(), 
            "I": inclination_slider.get(), 
            "O": raan_slider.get(), 
            "W": argument_periapsis_slider.get(), 
            "V": mean_anamoly_slider.get()  
        })

In [ ]:
csv_button = ttk.Button(right_frame, text="Download CSV", command=download_csv, style='TButton')
csv_button.pack()

## Step 9: Run the Tkinter Application

In [ ]:
root.mainloop()